<a href="https://colab.research.google.com/github/Naihell/TESTEPYSPARK-Confitec/blob/main/netflix/netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 45 kB/s 
     |████████████████████████████████| 199 kB 46.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=a520d82cc040901f4e28fed15b356425e024134d8baee167d87db6610dbd2f9b
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, current_timestamp, desc, lit, regexp_replace, to_date, to_timestamp
from pyspark.sql.types import TimestampType

# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# To fix date casting problems
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
# Check Spark Session Information
spark

In [3]:
drive_path = '/content/drive/MyDrive/confitec_test/OriginaisNetflix.parquet'

### Read raw file

In [31]:
df = spark.read.parquet(drive_path)

In [32]:
df.show(3)

+--------------------+---------------+---------------+---------+--------------------+-------------+--------------+----------+---------+---------+-------+------+-----+--------+--------------------+
|               Title|          Genre|    GenreLabels| Premiere|             Seasons|SeasonsParsed|EpisodesParsed|    Length|MinLength|MaxLength| Status|Active|Table|Language|         dt_inclusao|
+--------------------+---------------+---------------+---------+--------------------+-------------+--------------+----------+---------+---------+-------+------+-----+--------+--------------------+
|      House of Cards|Political drama|political,drama| 1-Feb-13|6 seasons, 73 epi...|            6|            73|42–59 min.|       42|       59|  Ended|     0|Drama| English|2021-03-16T21:20:...|
|       Hemlock Grove|Horror/thriller|horror,thriller|19-Apr-13|3 seasons, 33 epi...|            3|            33|45–58 min.|       45|       58|  Ended|     0|Drama| English|2021-03-16T21:20:...|
|Orange Is the 

#### 1. Transformar os campos "Premiere" e "dt_inclusao" de string para datetime.

In [33]:
df = df.withColumn('Premiere', to_date(col("Premiere"),"dd-MMM-yy")) \
      .withColumn('dt_inclusao', df.dt_inclusao.cast(TimestampType()))

#### 2. Ordenar os dados por ativos e gênero de forma decrescente, 0 = inativo e 1 = ativo, todos
com número 1 devem aparecer primeiro.

In [34]:
df = df.sort(df.Active.desc())

#### 3. Remover linhas duplicadas e trocar o resultado das linhas que tiverem a coluna "Seasons"
de "TBA" para "a ser anunciado".

In [35]:
df = df.withColumn('Seasons', regexp_replace(df.Seasons, 'TBA', 'a ser anunciado')).distinct()

#### 4. Criar uma coluna nova chamada "Data de Alteração" e dentro dela um timestamp.

In [36]:
df = df.withColumn('Data de Alteração', lit(current_timestamp()))

#### 5. Trocar os nomes das colunas de inglês para português, exemplo: "Title" para "Título"
(com acentuação).

In [37]:
df = df.withColumnRenamed('Title', 'Título') \
        .withColumnRenamed('Genre', 'Gênero') \
        .withColumnRenamed('GenreLabels', 'RótulosGênero') \
        .withColumnRenamed('Premiere', 'Estreia') \
        .withColumnRenamed('Seasons', 'Temporadas') \
        .withColumnRenamed('SeasonsParse', 'TemporadasAnalisadas') \
        .withColumnRenamed('EpisodesParsed', 'EpisódiosAnalisados') \
        .withColumnRenamed('Status', 'Status') \
        .withColumnRenamed('Active', 'Ativo') \
        .withColumnRenamed('Table', 'Grade') \
        .withColumnRenamed('Language', 'Idioma')

#### 6. Testar e verificar se existe algum erro de processamento do spark e identificar onde
pode ter ocorrido o erro.

In [38]:
# Tive um problema com o casting de data, mas resolvi com o
# spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

#### 7. Criar apenas 1 .csv com as seguintes colunas que foram nomeadas anteriormente "Title,
Genre, Seasons, Premiere, Language, Active, Status, dt_inclusao, Data de Alteração" as
colunas devem estar em português com header e separadas por ";".

In [45]:
output_path = '/content/drive/MyDrive/confitec_test/output'

selected_columns = df.select(
    'Título',
    'Gênero',
    'Temporadas',
    'Estreia',
    'Ativo',
    'Status',
    'dt_inclusao',
    'Data de Alteração'
)

# Ordenar as colunas novamente
selected_columns = selected_columns.sort(selected_columns.Ativo.desc())

selected_columns.coalesce(1).write.mode('overwrite').option('header', 'true').csv(output_path)

#### 8. Inserir esse .csv dentro de um bucket do AWS s3

In [40]:
# Vou ficar devendo, mas o procedimento é o mesmo.

In [46]:
selected_columns.show()

+--------------------+--------------------+--------------------+----------+-----+-------+--------------------+--------------------+
|              Título|              Gênero|          Temporadas|   Estreia|Ativo| Status|         dt_inclusao|   Data de Alteração|
+--------------------+--------------------+--------------------+----------+-----+-------+--------------------+--------------------+
| My First First Love|     Romantic comedy|1 season, 8 episodes|2019-04-18|    1|Renewed|2021-03-17 00:20:...|2022-11-14 01:53:...|
|      The Politician|              Comedy|     a ser anunciado|2019-09-27|    1|Renewed|2021-03-17 00:20:...|2022-11-14 01:53:...|
|      Super Monsters| childrens-animation|2 seasons, 16 epi...|2017-10-13|    1|Renewed|2021-03-17 00:20:...|2022-11-14 01:53:...|
|Kulipari: Dream W...| childrens-animation|1 season, 10 epis...|2018-11-20|    1|Pending|2021-03-17 00:20:...|2022-11-14 01:53:...|
|Inside the Real N...|         Docu-series|1 season, 3 episodes|2018-12-14| 